# Python Mcity OCTANE Socket.IO Interface
The API provides Socket.IO support for listening to a subscribable streams of events through web sockets.
Follow the steps below to test how to utilize the API through jQuery or other web javascript frameworks.
Viewing the source of this page will reveal the code running behind the examples shown here.

## Connect

Real time push event notification through the OCTANE Socket.IO interface utilizes namespaces and channels to control communication. As a client all communication will be done on the /octane namespace. This must be specified when connecting. Channels enable a client to subscribe to specific types of messages by joining or leaving a channel. On connection to the Mcity implementation the client is automatically joined to 2 channels (user, information). Namespaces and channel names are case sensitive. All channels and namespaces used in OCTANE are lower cased.

SocketIO Namespace: **/octane**

First let's load the required libraries and source our connection parameters from the environment. We'll setup a client and then after discussing authentication, connect.

In [1]:
import os
from dotenv import load_dotenv
import socketio
#Load environment variables
load_dotenv()
api_key = os.environ.get('MCITY_OCTANE_KEY', None) 
server = os.environ.get('MCITY_OCTANE_SERVER', 'http://localhost:5000')
namespace = "/octane"

#If no API Key provided, exit.
if not api_key:
    print ("No API KEY SPECIFIED. EXITING")
    exit()
    
#Create an SocketIO Python client.
sio = socketio.Client()

## Authentication

To use the Socket.IO interface a user must be authenticated. We support two ways of authentication.

1. Through a message to the server, after successful connection to socket.IO
2. Using a query Parameter

The preferred way to authenticate is through the message to server after connect.
To do this send  the event "auth" with a payload that has a key of x-api-key and value of the API key.

If utilizing the query parameter method, set X-API-KEY to your API key. The query parameter is not supported by all transport mechanisms. 
Query Parameter: **X-API-KEY=API_KEY_HERE**  

In [2]:
def send_auth():
    """
    Emit an authentication event with the API key.
    """
    sio.emit('auth', {'x-api-key': api_key}, namespace=namespace)

@sio.on('connect', namespace=namespace)
def on_connect():
    """
    Handle connection event and send authentication key as soon as it's recieved.
    """
    send_auth()

Now we have enough of a structure setup to allow us to connect and authenticate. We have no handlers for messages yet, but that will come next.

In [3]:
#Make connection.
sio.connect(server, namespaces=[namespace])

## Receive

Immediately after authentication  you'll start to receive different types of messages. 

The messages you can receive are documented as GET methods at this link: https://mcity.um.city/apidocs/#/WebSockets-Events  The payloads (with exception of user) are structured as documented at the same link. 

By parsing the payload data from received events we can utilize the facilities real time push notification for changes in infrastructure.

Our first setup will be to setup a message handler for intersection messages. We won't see any come across until we subscribe to the intersection channel, but that will happen next.

In [4]:
@sio.on('intersection_update', namespace=namespace)
def on_int_update(data):
    """
    Event fired when an intersection has been updated.
    """
    print('Int[U]:', data)

## Subscriptions

Channels control the type of events you will receive. To see a listing of channels supported and a description see the API docs section here: https://mcity.um.city/apidocs/#/WebSockets-Channels 

To get a listing of channels along with your currently subscribed channels emit a channel event with no payload. 
See documentation on possible event types here: https://mcity.um.city/apidocs/#/WebSockets-Events
The server will respond to our emit directly and list both the channels available to the clients and the channels we are currently subscribed to. 

Let's setup handlers for subscription and channel listing events and then join a channel.

On connection the server OCTANE automatically adds your user to the USER and FACILITY information channels and provides a listing of all channels currently subscribed. Since our event handlers were not in place at that time, we didn't see those events happen.

In [5]:
@sio.on('join', namespace=namespace)
def on_join(data):
    """
    Event fired when user joins a channel
    """
    print('Joining:', data)

@sio.on('leave', namespace=namespace)
def on_leave(data):
    """
    Event fired when user leaves a channel
    """
    print('Leaving:', data)
    
@sio.on('channels', namespace=namespace)
def on_channels(data):
    """
    Event fired when a user requests current channel information.
    """
    print('Channel information', data)

With all handlers in place, let's query channels possible, then join a channel, and leave.

In [6]:
sio.emit('channels', namespace='/octane')

Channel information {'channels': [{'name': 'facility', 'subscribed': True}, {'name': 'user', 'subscribed': True}, {'name': 'v2x', 'subscribed': False}, {'name': 'crosswalk', 'subscribed': False}, {'name': 'gate', 'subscribed': False}, {'name': 'railcrossing', 'subscribed': False}, {'name': 'signal', 'subscribed': False}, {'name': 'intersection', 'subscribed': False}]}


In [7]:
sio.emit('join', {'channel': 'crosswalk'}, namespace='/octane')

Joining: {'join': 'crosswalk'}


In [8]:
sio.emit('channels', namespace='/octane')

Channel information {'channels': [{'name': 'facility', 'subscribed': True}, {'name': 'user', 'subscribed': True}, {'name': 'v2x', 'subscribed': False}, {'name': 'crosswalk', 'subscribed': True}, {'name': 'gate', 'subscribed': False}, {'name': 'railcrossing', 'subscribed': False}, {'name': 'signal', 'subscribed': False}, {'name': 'intersection', 'subscribed': False}]}


In [9]:
sio.emit('leave', {'channel': 'crosswalk'}, namespace='/octane')

Leaving: {'leave': 'crosswalk'}


In [10]:
sio.emit('channels', namespace='/octane')

Channel information {'channels': [{'name': 'facility', 'subscribed': True}, {'name': 'user', 'subscribed': True}, {'name': 'v2x', 'subscribed': False}, {'name': 'crosswalk', 'subscribed': False}, {'name': 'gate', 'subscribed': False}, {'name': 'railcrossing', 'subscribed': False}, {'name': 'signal', 'subscribed': False}, {'name': 'intersection', 'subscribed': False}]}


## Communication:
OCTANE enables a user to communicate using Socket.IO with other clients. This is useful for implementing both process synchronization and just basic communication of users while testing.

This type of communication only occurs on the USER channel to which you are automatically subscribed on connection. For more information about this channel see the API documentation here: https://mcity.um.city/apidocs/#/WebSockets-Channels 

The format of these messages and payloads is left to the user. Anything submitted on this event, will be emitted to all other users currently connected to the API. 

Let's setup a handler for this and emit an event to the users channel.

In [11]:
@sio.on('user_message', namespace=namespace)
def on_user_message(data):
    """
    Event fired when a user sends a message to the user event channel.
    """
    print('User Message:', data)

In [12]:
sio.emit('user_message', {'message': 'Testing messaging'}, namespace='/octane')

User Message: {'message': 'Testing messaging'}


## Disconnecting

To disconnect from the server send a disconnect_request event.

If you submit a disconnect_event, the server will cleanup your subscriptions and then forcefully disconnect you.

Let's disconnect from OCTANE by emitting a disconnect_event.


In [13]:
@sio.on('disconnect_request', namespace=namespace)
def on_disconnect_request(data):
    """
    Event fired on disconnect request.
    """
    print('Ask to disconnect from server', data)
    
@sio.on('disconnect', namespace=namespace)
def on_disconnect_request():
    """
    Event fired on disconnec.
    """
    print('Disconnected from server')

In [14]:
sio.emit('disconnect_request', namespace='/octane')

Disconnected from server
